# Dynamic Tables: Incremental Refresh Demo

## 🎯 What You'll Learn

This notebook demonstrates:
1. **Incremental Refresh** - Processing only changed data, not entire datasets
2. **3-Tier Pipeline** - Data flowing through enrichment → integration → aggregation
3. **DOWNSTREAM Dependencies** - How automatic cascading works in production
4. **Manual vs Scheduled Refresh** - Critical distinction for understanding behavior

---

## ⚠️ Critical Concept: Manual vs Scheduled Refresh

**In this demo:**
- We manually refresh each tier to show the flow step-by-step
- Manual refresh **does NOT** cascade to DOWNSTREAM dependencies

**In production:**
- Scheduled refreshes **DO** cascade automatically to DOWNSTREAM dependencies
- Zero manual intervention required!

---

**🚀 Let's begin!**

## Step 1: Capture Baseline State

First, let's see the current row counts across all layers **before** we insert new data.

In [ ]:
-- Capture current state
CREATE OR REPLACE TEMPORARY TABLE demo_baseline AS
SELECT 
    'RAW: orders' AS layer,
    COUNT(*) AS row_count,
    1 AS sort_order
FROM automated_intelligence.raw.orders
UNION ALL
SELECT 
    'RAW: order_items',
    COUNT(*),
    2
FROM automated_intelligence.raw.order_items
UNION ALL
SELECT 'TIER 1: enriched_orders', COUNT(*), 3
FROM automated_intelligence.dynamic_tables.enriched_orders
UNION ALL
SELECT 'TIER 2: fact_orders', COUNT(*), 4
FROM automated_intelligence.dynamic_tables.fact_orders
UNION ALL
SELECT 'TIER 3: daily_metrics', COUNT(*), 5
FROM automated_intelligence.dynamic_tables.daily_business_metrics;

-- Display baseline
SELECT layer, row_count 
FROM demo_baseline 
ORDER BY sort_order;

## Step 2: Insert New Orders

⚠️ **NOTE**: The `generate_orders()` procedure has been removed.

Use **Snowpipe Streaming** to generate orders instead:
- See: `snowpipe-streaming-java/` or `snowpipe-streaming-python/`

For this demo, we'll work with existing orders in the system.

In [ ]:
-- ⚠️ generate_orders() procedure removed
-- Use Snowpipe Streaming instead (see snowpipe-streaming-java/ or snowpipe-streaming-python/)

-- For this demo: check existing order count
SELECT COUNT(*) as existing_orders FROM automated_intelligence.raw.orders;

### Verify RAW Data Updated

Let's confirm the new orders are in the RAW tables:

In [ ]:
SELECT 
    'RAW: orders' AS table_name,
    COUNT(*) AS current_count,
    COUNT(*) - (SELECT row_count FROM demo_baseline WHERE layer = 'RAW: orders') AS new_rows_added
FROM automated_intelligence.raw.orders
UNION ALL
SELECT 
    'RAW: order_items',
    COUNT(*),
    COUNT(*) - (SELECT row_count FROM demo_baseline WHERE layer = 'RAW: order_items') AS new_rows_added
FROM automated_intelligence.raw.order_items;

## Step 3: Refresh TIER 1 (Enrichment Layer)

Now we'll manually refresh the **Tier 1** dynamic tables:
- `enriched_orders` - Adds temporal dimensions and financial calculations
- `enriched_order_items` - Enriches order line items

### ⚠️ Important Note
This manual refresh will **NOT** automatically trigger Tier 2 or Tier 3.

In production, when Tier 1's **scheduled refresh** runs, it would automatically trigger Tier 2 (which has `TARGET_LAG = DOWNSTREAM`).

In [ ]:
-- Refresh Tier 1 tables
ALTER DYNAMIC TABLE automated_intelligence.dynamic_tables.enriched_orders REFRESH;
ALTER DYNAMIC TABLE automated_intelligence.dynamic_tables.enriched_order_items REFRESH;

SELECT '✅ Tier 1 refresh complete' AS status;

### Check Tier 1 Row Count

Tier 1 should now have the 500 new enriched orders:

In [ ]:
SELECT 
    'TIER 1: enriched_orders' AS layer,
    COUNT(*) AS current_count,
    COUNT(*) - (SELECT row_count FROM demo_baseline WHERE layer = 'TIER 1: enriched_orders') AS rows_added
FROM automated_intelligence.dynamic_tables.enriched_orders;

## Step 4: Refresh TIER 2 (Integration Layer)

Now we'll manually refresh **Tier 2**: `fact_orders`

This table joins enriched orders with enriched order items to create a denormalized fact table.

### 🔍 Why Manual Refresh Needed?
Even though Tier 2 has `TARGET_LAG = DOWNSTREAM`, the manual refresh of Tier 1 did **NOT** trigger it.

Only **scheduled refreshes** cascade to DOWNSTREAM dependencies!

In [ ]:
-- Refresh Tier 2
ALTER DYNAMIC TABLE automated_intelligence.dynamic_tables.fact_orders REFRESH;

SELECT '✅ Tier 2 refresh complete' AS status;

### Check Tier 2 Row Count

Tier 2 should now have new fact rows (one per order item):

In [ ]:
SELECT 
    'TIER 2: fact_orders' AS layer,
    COUNT(*) AS current_count,
    COUNT(*) - (SELECT row_count FROM demo_baseline WHERE layer = 'TIER 2: fact_orders') AS rows_added
FROM automated_intelligence.dynamic_tables.fact_orders;

## Step 5: Refresh TIER 3 (Aggregation Layer)

Finally, we'll manually refresh **Tier 3** aggregation tables:
- `daily_business_metrics` - Daily revenue, orders, customers
- `product_performance_metrics` - Product-level aggregations

These pre-compute metrics so queries are instant!

### 🔍 Why Manual Refresh Needed?
Same reason - manual refresh of Tier 2 did NOT cascade to Tier 3.

In production, Tier 3's scheduled refresh would be automatically triggered after Tier 2 completes.

In [ ]:
-- Refresh Tier 3
ALTER DYNAMIC TABLE automated_intelligence.dynamic_tables.daily_business_metrics REFRESH;
ALTER DYNAMIC TABLE automated_intelligence.dynamic_tables.product_performance_metrics REFRESH;

SELECT '✅ Tier 3 refresh complete' AS status;

## Step 6: Compare Before vs After

Let's see how the data propagated through all three tiers:

In [ ]:
-- Before vs After comparison
SELECT 
    b.layer,
    b.row_count AS before_count,
    CASE 
        WHEN b.layer = 'RAW: orders' THEN (SELECT COUNT(*) FROM automated_intelligence.raw.orders)
        WHEN b.layer = 'RAW: order_items' THEN (SELECT COUNT(*) FROM automated_intelligence.raw.order_items)
        WHEN b.layer = 'TIER 1: enriched_orders' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.enriched_orders)
        WHEN b.layer = 'TIER 2: fact_orders' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.fact_orders)
        WHEN b.layer = 'TIER 3: daily_metrics' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.daily_business_metrics)
    END AS after_count,
    CASE 
        WHEN b.layer = 'RAW: orders' THEN (SELECT COUNT(*) FROM automated_intelligence.raw.orders) - b.row_count
        WHEN b.layer = 'RAW: order_items' THEN (SELECT COUNT(*) FROM automated_intelligence.raw.order_items) - b.row_count
        WHEN b.layer = 'TIER 1: enriched_orders' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.enriched_orders) - b.row_count
        WHEN b.layer = 'TIER 2: fact_orders' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.fact_orders) - b.row_count
        WHEN b.layer = 'TIER 3: daily_metrics' THEN (SELECT COUNT(*) FROM automated_intelligence.dynamic_tables.daily_business_metrics) - b.row_count
    END AS rows_added,
    CASE
        WHEN b.layer = 'RAW: order_items' THEN '✅ RAW data'
        WHEN b.layer = 'RAW: orders' THEN '✅ RAW data'
        WHEN b.layer = 'TIER 3: daily_metrics' THEN '✅ Aggregated'
        ELSE '✅ Propagated'
    END AS status
FROM demo_baseline b
ORDER BY b.sort_order;

## Step 7: Prove Incremental Refresh

Let's look at the refresh history to prove that **incremental refresh** was used.

Key columns:
- `refresh_action`: Should be **INCREMENTAL** (not FULL)
- `seconds`: How long the refresh took
- `inserted`: Number of rows inserted 
- `efficiency`: Confirmation that only changed rows were processed

In [ ]:
-- Incremental refresh proof since n minutes ago
SET SINCE_MINUTES = -5;

SELECT 
    CASE 
        WHEN name LIKE '%enriched_orders' AND name NOT LIKE '%items' THEN 'T1: enriched_orders'
        WHEN name LIKE '%enriched_order_items' THEN 'T1: enriched_order_items'
        WHEN name LIKE '%fact_orders' THEN 'T2: fact_orders'
        WHEN name LIKE '%daily_business%' THEN 'T3: daily_metrics'
        WHEN name LIKE '%product_performance%' THEN 'T3: product_performance'
    END AS tier_table,
    refresh_action,
    DATEDIFF('second', refresh_start_time, refresh_end_time) AS seconds,
    statistics:numInsertedRows::INT AS inserted,
    statistics:numDeletedRows::INT AS deleted,
    CASE 
        WHEN refresh_action = 'INCREMENTAL' THEN '✅ Only changed rows'
        WHEN refresh_action = 'NO_DATA' THEN '✅ No changes detected'
        WHEN refresh_action = 'FULL' THEN '⚠️ Full table scan'
    END AS efficiency
FROM TABLE(INFORMATION_SCHEMA.DYNAMIC_TABLE_REFRESH_HISTORY(
    NAME_PREFIX => 'AUTOMATED_INTELLIGENCE.DYNAMIC_TABLES'
))
WHERE data_timestamp >= DATEADD('minute', $SINCE_MINUTES, CURRENT_TIMESTAMP())
ORDER BY 
    data_timestamp DESC,
    CASE 
        WHEN name LIKE '%enriched%' THEN 1
        WHEN name LIKE '%fact%' THEN 2
        ELSE 3
    END,
    name;

## Step 8: View Dynamic Table Configuration

Let's see the `TARGET_LAG` configuration for each table:
- **Tier 1**: `TARGET_LAG = '12 hours'` (time-based)
- **Tier 2 & 3**: `TARGET_LAG = DOWNSTREAM` (dependency-based)

In [ ]:
-- View dynamic table configuration
-- This shows target_lag, refresh_mode, and scheduling state for all dynamic tables
SHOW DYNAMIC TABLES IN SCHEMA automated_intelligence.dynamic_tables;

---

## 🎓 Key Takeaways

### What We Demonstrated:

1. **✅ Data Flow**
   - 500 orders inserted → RAW layer
   - Propagated through Tier 1 (enrichment) → Tier 2 (integration) → Tier 3 (aggregation)

2. **✅ Incremental Refresh**
   - Only processed ~500 new orders, not the entire dataset
   - Refreshes completed in seconds (see refresh history above)
   - Massive performance and cost savings!

3. **✅ Manual Cascade**
   - We manually refreshed each tier because **manual refresh does NOT cascade**
   - Manual refresh of Tier 1 → Did NOT trigger Tier 2
   - Manual refresh of Tier 2 → Did NOT trigger Tier 3

---

## 🚀 Production Deployment

### In Production, Everything is Automatic!

**You configure once (in DDL):**
```sql
-- Tier 1: Time-based refresh
CREATE DYNAMIC TABLE enriched_orders
  TARGET_LAG = '12 hours'
  ...

-- Tier 2: Dependency-based refresh
CREATE DYNAMIC TABLE fact_orders
  TARGET_LAG = DOWNSTREAM
  ...

-- Tier 3: Dependency-based refresh
CREATE DYNAMIC TABLE daily_metrics
  TARGET_LAG = DOWNSTREAM
  ...
```

**What happens automatically (forever):**
1. Every ~12 hours: Tier 1's **scheduled refresh** runs
2. Snowflake detects Tier 2 depends on Tier 1 → **automatically triggers** Tier 2 refresh
3. Snowflake detects Tier 3 depends on Tier 2 → **automatically triggers** Tier 3 refresh
4. All use incremental refresh → only process changes
5. Entire cascade completes in seconds

**You never have to:**
- Write orchestration code (no DAGs!)
- Manage dependencies manually
- Schedule jobs in external tools (no Airflow needed!)
- Monitor for failures in the cascade
- Manually trigger refreshes

**Deploy the DDL once, and it runs forever!** 🎯

---

### 🎉 Demo Complete!

- ✅ Incremental refresh processing
- ✅ 3-tier data pipeline
- ✅ Dynamic Tables with DOWNSTREAM dependencies
- ✅ Production-ready architecture with zero orchestration!

In [ ]:
-- Cleanup temporary table
DROP TABLE IF EXISTS demo_baseline;

SELECT '✅ Demo cleanup complete' AS status;